<a href="https://colab.research.google.com/github/marcos-da-luz-pinto/TSFMs/blob/main/Resultados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Carregar os arquivos de resultados
file_arima = 'RESULTADOS AutoARIMA'
file_chronos = 'RESULTADOS_Chronos.csv'
file_moirai = 'RESULTADOS_Moirai.csv'

df_arima = pd.read_csv(file_arima)
df_chronos = pd.read_csv(file_chronos)
df_moirai = pd.read_csv(file_moirai)

# Padronizar a identificação do modelo
df_arima['Modelo'] = 'AutoARIMA'

# Unificar todos os dados
df_all = pd.concat([df_arima, df_chronos, df_moirai], ignore_index=True)

# Filtrar pela configuração solicitada (C=120, H=24)
df_filtrado = df_all[(df_all['C'] == 120) & (df_all['H'] == 24)]

# Criar a Tabela Mestra para MASE (Mediana)
tabela_mase = df_filtrado.pivot_table(
    index='Variável',
    columns='Modelo',
    values='MASE (Mediana)'
).reset_index()

# Criar a Tabela Mestra para CRPS (Mediana)
tabela_crps = df_filtrado.pivot_table(
    index='Variável',
    columns='Modelo',
    values='CRPS (Mediana)'
).reset_index()

# Salvar os arquivos consolidados
tabela_mase.to_csv('TABELA_CONSOLIDADA_MASE_C120_H24.csv', index=False)
tabela_crps.to_csv('TABELA_CONSOLIDADA_CRPS_C120_H24.csv', index=False)

# Exibição dos resultados
print("--- TABELA CONSOLIDADA: MASE (C=120, H=24) ---")
print(tabela_mase.to_string(index=False))
print("\n--- TABELA CONSOLIDADA: CRPS (C=120, H=24) ---")
print(tabela_crps.to_string(index=False))

--- TABELA CONSOLIDADA: MASE (C=120, H=24) ---
                Variável  AutoARIMA  Chronos-Base  Moirai-Base
CUB Desp. Administrativa     0.0049        0.0074       0.0312
         CUB Equipamento     0.0090        0.0093       0.0311
              CUB Global     0.0469        0.0255       0.3312
         CUB Mão de Obra     0.0093        0.0077       0.1578
            Cub Material     0.0905        0.0311       0.4384
        INCC- DI_var_mes     0.0006        0.0006       0.0009
          Taxa_SELIC_avg     0.0114        0.0204       0.0361

--- TABELA CONSOLIDADA: CRPS (C=120, H=24) ---
                Variável  AutoARIMA  Chronos-Base  Moirai-Base
CUB Desp. Administrativa     1.0770        1.8504       7.5663
         CUB Equipamento     0.6062        0.5834       2.4923
              CUB Global    57.4585       39.4780     479.0342
         CUB Mão de Obra    11.6624       11.7250     216.2850
            Cub Material    42.9238       18.7799     241.6507
        INCC- DI_var_me

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare

# Carregamento dos dados
# Certifique-se de que os nomes dos arquivos estão corretos no seu ambiente
file_arima = 'RESULTADOS AutoARIMA'
file_chronos = 'RESULTADOS_Chronos.csv'
file_moirai = 'RESULTADOS_Moirai.csv'

df_arima = pd.read_csv(file_arima)
df_chronos = pd.read_csv(file_chronos)
df_moirai = pd.read_csv(file_moirai)

# Padronização e União
df_arima['Modelo'] = 'AutoARIMA'
df_all = pd.concat([df_arima, df_chronos, df_moirai], ignore_index=True)

def realizar_analise_estatistica(df, metrica):
    """
    Realiza o Teste de Friedman e calcula os postos médios e a Diferença Crítica.
    """
    # Criar tabela pivot: cada linha é um "bloco" (Variável + C + H)
    pivot = df.pivot_table(
        index=['Variável', 'C', 'H'],
        columns='Modelo',
        values=metrica
    ).dropna()

    # Teste de Friedman
    # H0: Não há diferença entre os modelos
    stat, p_value = friedmanchisquare(
        pivot['AutoARIMA'],
        pivot['Chronos-Base'],
        pivot['Moirai-Base']
    )

    # Cálculo dos Ranks (Postos) - Menor erro ganha o melhor rank (1)
    ranks = pivot.rank(axis=1, ascending=True)
    avg_ranks = ranks.mean().sort_values()

    # Cálculo da Diferença Crítica (CD) para o Teste de Nemenyi
    # Fórmula: CD = q_alpha * sqrt( (k * (k + 1)) / (6 * n) )
    k = 3  # Número de modelos
    n = len(pivot)  # Número de blocos (42 no seu caso)
    q_alpha = 2.344 # Valor crítico para k=3 e alpha=0.05

    cd = q_alpha * np.sqrt((k * (k + 1)) / (6 * n))

    return {
        'metrica': metrica,
        'estatistica': stat,
        'p_valor': p_value,
        'ranks_medios': avg_ranks,
        'cd': cd,
        'n': n
    }

# Execução para MASE e CRPS
resultados_mase = realizar_analise_estatistica(df_all, 'MASE (Mediana)')
resultados_crps = realizar_analise_estatistica(df_all, 'CRPS (Mediana)')

# Exibição dos Resultados
for res in [resultados_mase, resultados_crps]:
    print(f"\n--- ANÁLISE ESTATÍSTICA: {res['metrica']} ---")
    print(f"Número de Blocos (n): {res['n']}")
    print(f"Estatística de Friedman: {res['estatistica']:.4f}")
    print(f"p-valor: {res['p_valor']:.10f}")
    print(f"Diferença Crítica (CD) de Nemenyi: {res['cd']:.4f}")
    print("Ranks Médios (Postos):")
    print(res['ranks_medios'])
    print("-" * 40)


--- ANÁLISE ESTATÍSTICA: MASE (Mediana) ---
Número de Blocos (n): 42
Estatística de Friedman: 55.7939
p-valor: 0.0000000000
Diferença Crítica (CD) de Nemenyi: 0.5115
Ranks Médios (Postos):
Modelo
Chronos-Base    1.392857
AutoARIMA       1.690476
Moirai-Base     2.916667
dtype: float64
----------------------------------------

--- ANÁLISE ESTATÍSTICA: CRPS (Mediana) ---
Número de Blocos (n): 42
Estatística de Friedman: 60.0476
p-valor: 0.0000000000
Diferença Crítica (CD) de Nemenyi: 0.5115
Ranks Médios (Postos):
Modelo
Chronos-Base    1.50000
AutoARIMA       1.52381
Moirai-Base     2.97619
dtype: float64
----------------------------------------
